# `Группировка данных и оконные функции  PostgreSQL`

<img src='data/img/pglogo.png' width=700>

____

### <a id=0>Содержание</a>
- [Исходные данные](#1)
- [Задача](#2)
- [Решения](#2)


___
## <center> <a id=1>Исходные данные</a>

Дано два csv-файла с данными о клиентах :

- `customer.csv`

|Поле |Описание |
|:--|:--|
|customer_id  |id клиента |
| first_name  | 	имя клиента |
| last_name  | 	фамилия клиента |
| gender  |пол  |
|  DOB | 	дата рождения |
| job_title  | профессия |
| job_industry_category |	сфера деятельности  |
| wealth_segment  |	сегмент благосостояния  |
|deceased_indicator   | 	флаг актуального клиента |
| owns_car | флаг наличия автомобиля |
|address   | 	адрес проживания |
| postcode  |	почтовый индекс  |
| state  | штат |
|country   | страна проживания |
| property_valuation |  оценка имущества|

- `transaction.csv`

|Поле |Описание |
|:--|:--|
|transaction_id | id транзакции|
|product_id | 	id продукта|
|customer_id | id клиента|
|transaction_date |	дата транзакции |
|online_order |флаг онлайн-заказа |
| order_status| 	статус транзакции|
|brand | бренд|
| product_line| 	линейка продуктов|
| product_class|	класс продукта |
|product_size | 	размер продукта|
|list_price |	цена |
|standard_cost |	стандартная стоимость |


___
## <center> <a id=2>Задача</a>

$\boxed{1}$ Создать таблицы со следующими структурами и загрузить данные из csv-файлов

$\boxed{2}$ Выполнить следующие запросы :

- Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

- Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

- Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

- Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат

- Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

- Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

- Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

___
## <center> <a id=3>Решение</a>

___
###  <a id=3>Создание таблиц</a>

In [ ]:
create table customer (
"customer_id" int4,
"first_name" varchar(50),
"last_name" varchar(50),
"gender" varchar(30),
"DOB" varchar(50),
"job_title" varchar(50) ,
"job_industry_category" varchar(50) ,
"wealth_segment"  varchar(50),
"deceased_indicator"  varchar(50),
"owns_car"  varchar(30),
"address"  varchar(50),
"postcode"  varchar(30),
"state"  varchar(30),
"country"  varchar(30),
"property_valuation"  int4
);

create table transaction (
"transaction_id"  int4,
"product_id"  int4,
"customer_id"  int4,
"transaction_date"  varchar(30),
"online_order"  varchar(30),
"order_status"  varchar(30),
"brand"  varchar(30),
"product_line"  varchar(30),
"product_class"  varchar(30),
"product_size"  varchar(30),
"list_price"  float4,
"standard_cost"  float4
);

___
### <a id=4>Выполнение запросов</a>

___
- Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества

In [ ]:
select job_industry_category , count(job_industry_category) customers_count 
from customer c 
group by job_industry_category
order by count(job_industry_category)

___
- Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [ ]:
select 
    sum(list_price) as "Transactions sum"
    ,date_trunc('month', transaction_date ::date) as month
    , job_industry_category
from  "transaction" t 
left join customer c
on c.customer_id = t.customer_id 
where job_industry_category is not null
group by month, job_industry_category
order by month , job_industry_category

___
- Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

In [ ]:
select 
	brand , count(brand) "Brand count for customers in IT"
from "transaction" t 
left join customer c 
on c.customer_id = t.customer_id 
where online_order = 'True' and job_industry_category = 'IT' and order_status = 'Approved'
group by brand

___
- Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат

Используя `GROUP BY` :

In [ ]:
select 
customer_id
,sum(list_price) 
, max(list_price)
, min(list_price)
, count(list_price) 
from "transaction" t 
group by customer_id 
order by sum(list_price) desc, count(list_price) desc

Используя `оконные функции` :

In [ ]:
select 
customer_id
,sum(list_price) over (partition by customer_id) as "SUM"
,max(list_price) over (partition by customer_id)
,min(list_price) over (partition by customer_id)
,count(list_price) over (partition by customer_id) as "COUNT"
from "transaction" t 
order by "SUM" desc, "COUNT" desc


___
- Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

`Максимальная` сумма транзакций

In [ ]:
with groupped as (select
c.first_name as "NAME"
, c.last_name as "LAST"
, sum(t.list_price) as "SUM_PRICE"
from "transaction" t 
left join customer c 
on t.customer_id = c.customer_id 
group by c.first_name, c.last_name
order by "SUM_PRICE" desc )
select 
"NAME"
, "LAST"
, "SUM_PRICE"
from groupped
where "SUM_PRICE" = (select "SUM_PRICE" from groupped limit 1)

`Минимальная` сумма транзакций

In [ ]:
with groupped as (select
c.first_name as "NAME"
, c.last_name as "LAST"
, sum(t.list_price) as "SUM_PRICE"
from "transaction" t 
left join customer c 
on t.customer_id = c.customer_id 
group by c.first_name, c.last_name
order by "SUM_PRICE"  )
select 
"NAME"
, "LAST"
, "SUM_PRICE"
from groupped
where "SUM_PRICE" = (select "SUM_PRICE" from groupped limit 1)

___
- Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

$\boxed{1}$ Создаем `представление` :

In [ ]:
create view  table_view as
select 
first_value (transaction_date) over (partition by customer_id order by transaction_date) as "FIRST"
, *
from "transaction" t 

$\boxed{2}$ Используем `представление` для запроса:

In [ ]:
select * from table_view
where transaction_id in(select transaction_id from "transaction" where transaction_date = "FIRST")

___
- Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) 

$\boxed{1}$ Создаем `временную таблицу` :

In [ ]:
create temp table "TEMP_TABLE" as
select 
c.first_name
, c.last_name
, c.job_title
, t.transaction_date
, t.customer_id
, lead (transaction_date) over (partition by t.customer_id order by  t.transaction_date ::date) as "NEXT"
from "transaction" t 
left join customer c 
on t.customer_id = c.customer_id

$\boxed{2}$ Создаем `табличное выражение` :

In [ ]:
with "CTE" as (
select
first_name
,last_name
,job_title
, ("NEXT" ::date - transaction_date ::date ) as "INTER"
, "NEXT"
, transaction_date
from "TEMP_TABLE")
select 
first_name
,last_name
,job_title from "CTE"
where "INTER" = (select max("INTER") from "CTE")